In [1]:
import pandas as pd
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
IMAGES_PATH = "/home/phoenix/ehsan/projects/datasets/GLAMI/images"
DATA_SAVE_DIR = "data/"
MAPP_SAVE_DIR = "data/"
IMAGES_EXT = ".jpg"

In [3]:
df = pd.read_csv("/home/phoenix/ehsan/projects/datasets/GLAMI/GLAMI-1M-train.csv")
df.drop(columns=['item_id', 'name', 'geo', 'description', 'category', 'label_source'], inplace=True, axis=1)
df['label'] = pd.Categorical(df.category_name).codes
df["image_id"] = df["image_id"].astype(int)

In [4]:
df.head()

,image_id,category_name,label
0,0,mens-watches,111
1,3,mens-watches,111
2,5,womens-earrings,139
3,7,mens-boots,58
4,8,womens-bathrobes,128


In [5]:
image_paths = list(Path(IMAGES_PATH).rglob("*.jpg"))

df_paths = pd.DataFrame({
    "image_id": [int(p.stem) for p in image_paths],
    "path": [str(p) for p in image_paths]
})

In [6]:
df_paths.head()

,image_id,path
0,109803,/home/phoenix/ehsan/projects/datasets/GLAMI/im...
1,109917,/home/phoenix/ehsan/projects/datasets/GLAMI/im...
2,109517,/home/phoenix/ehsan/projects/datasets/GLAMI/im...
3,109860,/home/phoenix/ehsan/projects/datasets/GLAMI/im...
4,1077589,/home/phoenix/ehsan/projects/datasets/GLAMI/im...


In [7]:
df = df_paths.merge(df, on="image_id", how="inner")
df_map_cat = pd.DataFrame({"category": df.category_name.unique(), "label": df["label"].unique()})
df.drop(columns=["image_id", "category_name"], inplace=True, axis=1)

In [8]:
df.head()

,path,label
0,/home/phoenix/ehsan/projects/datasets/GLAMI/im...,134
1,/home/phoenix/ehsan/projects/datasets/GLAMI/im...,134
2,/home/phoenix/ehsan/projects/datasets/GLAMI/im...,189
3,/home/phoenix/ehsan/projects/datasets/GLAMI/im...,134
4,/home/phoenix/ehsan/projects/datasets/GLAMI/im...,175


In [9]:
# replace /home/phoenix/ehsan/projects/datasets/GLAMI/images/ with data/
# df["path"] = df["path"].str.replace("/home/phoenix/ehsan/projects/datasets/GLAMI/images/", "data/") 

In [10]:
# df.head()

In [11]:
df_pyarrow = pa.Table.from_pandas(df)
pq.write_table(df_pyarrow, Path(DATA_SAVE_DIR).joinpath("data.parquet"))

df_map_cat.to_csv(Path(MAPP_SAVE_DIR).joinpath("mapping.csv"), index=False)

In [13]:
parquet_test = pq.read_table(Path(DATA_SAVE_DIR).joinpath("data.parquet"))
df_test = parquet_test.to_pandas()
df_test.head()


,path,label
0,/home/phoenix/ehsan/projects/datasets/GLAMI/im...,134
1,/home/phoenix/ehsan/projects/datasets/GLAMI/im...,134
2,/home/phoenix/ehsan/projects/datasets/GLAMI/im...,189
3,/home/phoenix/ehsan/projects/datasets/GLAMI/im...,134
4,/home/phoenix/ehsan/projects/datasets/GLAMI/im...,175
